## Описание корпуса и стандарта разметки

В качестве корпуса были взяты отрывки из "Руслана и Людмилы" А.С. Пушкина.

#### Сложности этого текста для автоматической разметки:
1. Не всегда привычный порядок слов (потому что стихи)
2. Устаревшие грамматические формы ("нетерпенья", "боле", "решетчату", "щипля", "объемлет" и т.д.)
3. Устаревшие слова / варианты слов ("младые", "средь", "гридница", "брашна")
4. Формы с дефисом ("старца-колдуна", "Владимир-солнце", "кто-то", "теперь-то")
5. Имена собственные, в основном нестандартные (Ратмир, Фарлаф, Рогдай, Руслан, Баян)
6. Омонимия, связанная с отсутсвием ё ("слез", "все") 
7. Частеречная омонимия ("то", "смущённый", "перлов" и т.д.)


#### Выбранный тагсет
Пожалуй, я воспользуюсь тагсетом, предоставленным организаторами соревнования Dialog: 
- S существительные
- A прилагательные 
- V глаголы, в том числе причастия и деепричастия 
- PR предлоги 
- CONJ союзы
- PRO местоимения
- ADV сборная категория, включающая прочие несклоняемые слова: наречия, вводные слова, частицы, междометия

Местоимения (размечены как PRO), числительные (размечены как NUM), слова категории состояния (размечены как CAT), наречия и прилагательные в сравнительной степени (размечены как COMP) и составные предлоги и союзы так же будут исключены из оценки.

Если форма с **дефисом** была разделена, то оценивалось соответствие тега **последней части** (той, которая после дефиса). В нашем стандарте все формы с дефисом остались слитны и получили единый тег.

In [9]:
from pprint import pprint

In [59]:
raw_text = ''
with open('pushkin.txt', 'r', encoding='utf-8') as raw_file:
    raw_text = raw_file.read()

# Как показала практика прошлого года, текст лучше скармливать
# со знаками препинания. Но перенос строки мы всё же
# заменим на пробел

raw_text = raw_text.replace('\n', ' ')

In [100]:
answers = []
with open('pushkin POS.txt', 'r', encoding='utf-8') as answer_file:
    words = answer_file.readlines()
    for word in words:
        answers.append(word.strip('\n').split(' '))

## Разметить текст тремя POS-тэггерами + превести к стандартному виду

### pymorphy

In [61]:
import nltk
from nltk.tokenize import word_tokenize

# сначала лемматизируем

tokenized = word_tokenize(raw_text)
punct = '...,—;:'

In [74]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

# теперь частеречная разметка

pymorphy_answer_raw = []

for token in tokenized:
    if token not in punct:
        ama = morph.parse(token)[0].tag.POS
        pymorphy_answer_raw.append([token, ama])

In [76]:
# приведём разметку к единому стандарту

pm_tags = {'NOUN':'S', 'ADJF':'A', 'PRCL':'ADV', 
           'VERB':'V', 'PRTF':'V', 'CONJ':'CONJ',
           'PREP':'PR', 'GRND':'V', 'ADJS':'ADJ',
           'NPRO':'PRO', 'NUMR':'NUM', 'COMP':'COMP',
           'INTJ':'ADV', 'ADVB':'ADV', 'INFN':'V'}

pymorphy_answer = []
for par in pymorphy_answer_raw:
    pymorphy_answer.append([par[0], pm_tags[par[1]]])

### Mystem

In [13]:
from pymystem3 import Mystem
m = Mystem()

Installing mystem to C:\Users\Алина/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.0-win7-32bit.zip


In [14]:
ana = m.analyze(raw_text)

In [79]:
mystem_answer_raw = []

for an in ana:
    if an.get('analysis') != None:
        pos = an['analysis'][0]['gr'].split('=')[0].split(',')[0]
        mystem_answer_raw.append([an['text'], pos])

In [82]:
# приведём разметку к единому стандарту

ms_tags = {'APRO':'PRO', 'SPRO':'PRO', 
           'ADVPRO':'PRO', 'PART':'ADV',
           'INTJ':'ADV'}

mystem_answer = []
for mar in mystem_answer_raw:
    mystem_answer.append([mar[0], ms_tags.get(mar[1], mar[1])])

### spacy
в другой тетрадке, т.к. jupyter notebook не запускал эту библиотеку, а google colab - mystem

## Оценка accuracy

In [101]:
print(len(pymorphy_answer))
print(len(mystem_answer))
print(len(answers))

223
226
223


In [108]:
out_tags = ['PRO', 'NUM', 'CAT', 'COMP'] # не учитываем

In [109]:
pm_t_score = 0
total = 0

for i in range(0, 223):
    if answers[i][1] not in out_tags:
        total += 1
        if answers[i][1] == pymorphy_answer[i][1]:
             pm_t_score += 1

print('Accuracy for pymorphy: ')
print(str(pm_t_score/total))

Accuracy for pymorphy: 
0.923469387755102


In [112]:
gap = 0
ms_t_score = 0 # количество правильных

for i, a in enumerate(answers):
    # сверяем, что мы смотрим на одно и то же 
    # mystem разделил 3 формы с пробелами ()
    if a[0] != mystem_answer[i+gap][0]:
        gap += 1
        
    # сверяем сами теги
    if a[1] not in out_tags:
        if a[1] == mystem_answer[i+gap][1]:
            ms_t_score += 1

print('Accuracy for mystem: ')
print(str(ms_t_score/total))

Accuracy for mystem: 
0.9285714285714286


Вывод: pymorphy и mystem справились примерно одинаково очень хорошо.